In [2]:
from vnstock import *
from datetime import date, datetime, timedelta
from vnstock.chart import *
import plotly.io as pio
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.gridspec import GridSpec
import matplotlib.ticker as mticker
from mplfinance.original_flavor import candlestick_ochl
import math
from matplotlib.dates import date2num
from matplotlib.dates import DateFormatter, MonthLocator
from mplfinance.original_flavor import candlestick_ohlc

In [3]:
stock_codes = ['VHC', 'VCG', 'HSG', 'SSI', 'MWG', 'GIL', 'DXG']

In [4]:
def combine_columns(row):
    return f"{row['change']} ({row['%_change']}%)"

In [5]:
# Lấy dữ liệu cổ phiếu

def real_time_stock(stock_code):
    time_stamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    stock_hist = stock_historical_data(stock_code, "2022-01-01",
                                         str(date.today()), "1D", "stock")
    stock_hist["%_change"] = round(((stock_hist['close'] - stock_hist['close'].shift(1))
                                    / stock_hist['close'].shift(1)) * 100, 2)
    stock_hist['change'] = (stock_hist['close'] - stock_hist['close'].shift(1))
    stock_hist['latest_change'] = stock_hist.apply(combine_columns, axis=1)
    data = stock_hist.iloc[-1]
    # return (str(time_stamp), data['close'], data['volume'], data['%_change'], data['change'])
    return stock_hist

In [6]:

vhc = real_time_stock('VHC')
index_with_nan = vhc.index[vhc.isnull().any(axis=1)]
vhc.drop(index_with_nan, inplace=True)
vhc.to_csv('vhc.csv', index=False, header=True)
vhc

,time,open,high,low,close,volume,ticker,%_change,change,latest_change
1,2022-01-05,61750,65550,61270,65550,2390700,VHC,6.99,4280.0,4280.0 (6.99%)
2,2022-01-06,66720,67110,64580,65260,1103900,VHC,-0.44,-290.0,-290.0 (-0.44%)
3,2022-01-07,65260,66530,65260,65360,603500,VHC,0.15,100.0,100.0 (0.15%)
4,2022-01-10,65450,65550,61850,62630,1294300,VHC,-4.18,-2730.0,-2730.0 (-4.18%)
5,2022-01-11,61950,62820,59900,60390,972700,VHC,-3.58,-2240.0,-2240.0 (-3.58%)
...,...,...,...,...,...,...,...,...,...,...
447,2023-10-19,72800,73100,70100,70500,974100,VHC,-3.03,-2200.0,-2200.0 (-3.03%)
448,2023-10-20,70500,74800,68000,74800,921600,VHC,6.10,4300.0,4300.0 (6.1%)
449,2023-10-23,74400,74800,71600,72200,471200,VHC,-3.48,-2600.0,-2600.0 (-3.48%)
450,2023-10-24,73000,73000,71000,72500,391500,VHC,0.42,300.0,300.0 (0.42%)


In [7]:
def figure_design(ax):
    ax.set_facecolor('#091217')
    ax.tick_params(axis='both', labelsize=9, colors='white')
    ## ax.ticklabel_format(useOffset=False)
    ax.spines['bottom'].set_color('#808080')
    ax.spines['top'].set_color('#808080')
    ax.spines['left'].set_color('#808080')
    ax.spines['right'].set_color('#808080')

In [8]:
def read_data_ohlc(filepath):

    ## Đọc file
    df = pd.read_csv(filepath, index_col='time', parse_dates=['time'])
    df = df[['ticker', 'open', 'high', 'low', 'close' ,'latest_change', 'volume']]
    df['date'] = df.index
    df.index = pd.DatetimeIndex(df.index)
    df['MA20'] = df['close'].rolling(20).mean()
    df['MA50'] = df['close'].rolling(50).mean()
    df['MA200'] = df['close'].rolling(200).mean()

    ## Lấy giá đóng cửa, % thay đổi, khối lượng
    latest_info = df.iloc[-1, :]
    latest_price = latest_info['close']
    latest_change = str(latest_info['latest_change']) if (latest_info['latest_change'][0] == '-') else '+' + str(latest_info['latest_change'])
    latest_volume = latest_info['volume']

    return df, latest_price, latest_change, latest_volume

In [9]:
read_data_ohlc('vhc.csv')[0]

,ticker,open,high,low,close,latest_change,volume,date,MA20,MA50,MA200
time,,,,,,,,,,,
2022-01-05,VHC,61750,65550,61270,65550,4280.0 (6.99%),2390700,2022-01-05,NaN,NaN,NaN
2022-01-06,VHC,66720,67110,64580,65260,-290.0 (-0.44%),1103900,2022-01-06,NaN,NaN,NaN
2022-01-07,VHC,65260,66530,65260,65360,100.0 (0.15%),603500,2022-01-07,NaN,NaN,NaN
2022-01-10,VHC,65450,65550,61850,62630,-2730.0 (-4.18%),1294300,2022-01-10,NaN,NaN,NaN
2022-01-11,VHC,61950,62820,59900,60390,-2240.0 (-3.58%),972700,2022-01-11,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2023-10-19,VHC,72800,73100,70100,70500,-2200.0 (-3.03%),974100,2023-10-19,79240.0,77626.0,67070.495
2023-10-20,VHC,70500,74800,68000,74800,4300.0 (6.1%),921600,2023-10-20,78790.0,77562.0,67100.495
2023-10-23,VHC,74400,74800,71600,72200,-2600.0 (-3.48%),471200,2023-10-23,78500.0,77500.0,67112.495


In [10]:
## Create Open High Low Close
data, latest_price, latest_change, latest_volume = read_data_ohlc('vhc.csv')

candle_counter = range(len(data['open']) - 1)
ohlc = []
for  candle in candle_counter:
    item = date2num(data['date'][candle]), data['open'][candle],\
            data['high'][candle], data['low'][candle],\
            data['close'][candle]
    ohlc.append(item)

C:\Users\PC\AppData\Local\Temp\ipykernel_15196\3308182266.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  item = date2num(data['date'][candle]), data['open'][candle],\
C:\Users\PC\AppData\Local\Temp\ipykernel_15196\3308182266.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  data['high'][candle], data['low'][candle],\
C:\Users\PC\AppData\Local\Temp\ipykernel_15196\3308182266.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  

In [11]:
# Vẽ biểu đồ

def real_time_chart(stock_code):
    stock = stock_historical_data(stock_code, "2022-01-01", str(date.today()), "1D", "stock")
    fig = candlestick_chart(stock, ma_periods=[20, 50, 200], show_volume=True, reference_period=300, figure_size=(15, 8), 
                        title=f'{stock_code} - Candlestick Chart with MA and Volume', x_label='Date', y_label='Price', 
                        colors=('lightgray', 'gray'), reference_colors=('black', 'blue'))
    # pio.write_image(fig, 'plot.png')
    fig.show()

In [12]:
# real_time_chart('VHC')
stock = stock_historical_data('VHC', "2022-01-01", str(date.today()), "1D", "stock")
stock

,time,open,high,low,close,volume,ticker
0,2022-01-04,61270,62440,61170,61270,610400,VHC
1,2022-01-05,61750,65550,61270,65550,2390700,VHC
2,2022-01-06,66720,67110,64580,65260,1103900,VHC
3,2022-01-07,65260,66530,65260,65360,603500,VHC
4,2022-01-10,65450,65550,61850,62630,1294300,VHC
...,...,...,...,...,...,...,...
447,2023-10-19,72800,73100,70100,70500,974100,VHC
448,2023-10-20,70500,74800,68000,74800,921600,VHC
449,2023-10-23,74400,74800,71600,72200,471200,VHC
450,2023-10-24,73000,73000,71000,72500,391500,VHC


In [13]:
fig = candlestick_chart(stock, ma_periods=[20, 50, 200], show_volume=True, reference_period=300, figure_size=(15, 8), 
                    title=f'VHC - Candlestick Chart with MA and Volume', x_label='Date', y_label='Price', 
                    colors=('lightgray', 'gray'), reference_colors=('black', 'blue'))
fig.show()